In [11]:
#imports
import tensorflow as tf
import dataset
from tensorflow import keras
import pandas as pd
import numpy as np
# /!\ add your models here ! 
import models.classifiers.classifier1 as classifier1
import models.classifiers.classifier_thomas as classifier_thomas
# /!\ add your model in this dict ! 
models = {
    # this is to train models automatically
    "classifier1":classifier1,
    "classifier_thomas":classifier_thomas,
}

# /!\ add your analysis modules here ! 
import analysis.metrics as metrics
import analysis.accuracy as accuracy
import analysis.precision as precision
import analysis.recall as recall
import analysis.confusion_matrix as confusion_matrix


In [20]:

#set the dict here
hyperparams = {
    "train_ratio": 0.5,
    "val_ration": 0.3,
    "test_ration": 0.2,
    "batch_size": 32,
    'nb_epochs':10, #10
    'learning_rate': 3*1e-3, #3*1e-3
    'archi': 'convo',  # or 'convo'
    'kernel_size': (3, 3),
    'activation': 'relu',
    'nb_targets': 10,
    'nb_layers': 3,
    'add_pooling': True,
    'pooling_size': (2, 2),
    'nb_units': [32,64,128],
    'optimizer': keras.optimizers.Adam,
    'loss': 'sparse_categorical_crossentropy',  # 'metrics' : ['accuracy'],
}


model_params = {
    "saved":False,
    "to save":False,
    "classifier": "classifier_thomas",
    "hyperparams": hyperparams,
    "wm": None,
}

data_params= {
    "dataset": "cifar-10",
    "set": "train",
    "n": 40000,
    "seed":42 
}
data_params_test= {
    "dataset": "cifar-10",
    "set": "test",
    "n": 3000,
    "seed":42 
}

#comment stocker le résultat 
trigger_params = {
"n" : 50,
"nb_app_epoch":100,
"variance":5,
"from": 'dataset',
"noise":True,
"seed":2
}


model_params2 = {
    "saved": None,
    "to save": "integration_test2",
    "classifier": "classifier_thomas",
    "hyperparams": hyperparams,
    "wm": trigger_params,
}
analysis_params =  {
    #"processes": [("train", (model_params,data_params)),("wm", (model_params2,trigger_params,data_params))],
    "processes": [("train", (model_params,data_params)),("train", (model_params,data_params))],
    "analysis": [("metrics", (data_params_test,False)),
                    ("accuracy", (data_params_test,False)),
                    ("precision", (data_params_test,False)),
                    ("recall", (data_params_test,False)),
                    ("confusion_matrix", (data_params_test, False)),
                    ("accuracy", (data_params_test,trigger_params)),
                    ("confusion_matrix", (data_params_test, trigger_params))]
}


In [21]:

#from finetuning import fineT
# /!\ add your model in this dict ! 
analysis = {
    # this is to use analysis automatically
    "metrics":metrics,
    "accuracy":accuracy,
    "precision":precision,
    "recall":recall,
    "confusion_matrix":confusion_matrix
}
metrics_value={}
# results={}
# main functions
def main(model_params:dict,data_params:dict,analysis_params:dict=None) -> str:
    ''' This is the main function.
    It will create a model (training or loading from file), 
    watermark it, process it through some attacks 
    and then analyse its behaviour over a test set.
    Prints and returns the results.
    '''
    #model_setup
    model = None
    model = model_setup(model_params,data_params,model)
    #process
    model = process(model, analysis_params, data_params)
    #analysis
    results=result(model, analysis_params, data_params)
    # n=fineT.shape[0]
    # fineT[f'model {n+1}']=model_params['hyperparams']
    #fineT[f'model {n+1}']=[model_params,result(model, analysis_params, data_params)]
    # do the results  

def model_setup(model_params:dict,data_params:dict,model) -> tf.keras.Model:
    '''
    Function responsible for reading model_params dict
    data_params is needed to have the size of the images (will not train)

    train models from ./models/classifiers or load them from ./models/saved

    Please refer to nomenclature.md on how to fill out model_params
    '''
    #saved model model_params has no need to be passed to load a model
    # getting model
    model = models[model_params["classifier"]].get_model(model_params, data_params, model)
    try: model is not None
    except: raise Exception("Model is None. Verify loading names and parameters.")

    return model

def process(model: tf.keras.Model, analysis_params:dict , data_params:dict) -> tf.keras.Model:
    '''
    This is the processing part of main.py 
    where the model is subjected to changes (watermarking, attacks, retrain, ...)

    trigger_params dict is used to get the trigger set from triggerset.py
    data_params dict is used to access the dataset to train the model further and try to remove the WM

    Please refer to nomenclature.md on how to fill out the dictionaries
    '''
    # when implementing multiple analysis steps: loop over them
    # this is done by looping over analysis_params["processes"]
    # the dataset may be taken from the function argument or from the analysis_params dict
    # or just use classifiers module.train which will accept a model already trained ... 
   
    for process, p_args in analysis_params["processes"]:
        print(process)
        #train
        if process == "train":
            model_params, data_params1 = p_args
            if data_params1 != None: data_params = data_params1
            model = models[model_params["classifier"]].get_model(model_params, data_params, model)
        #watermark
        if process == "wm":
            model_params,trigger_params, data_params1 = p_args
            if data_params1 != None: data_params = data_params1
            if trigger_params!= None: model_params["wm"]=trigger_params
            model = models[model_params["classifier"]].get_model(model_params, data_params, model)
    return model
    

def result(model: tf.keras.Model, analysis_params:dict, data_params:dict) -> None:
    '''
    This is the analysis part of main.py. 
    where the processed model is assessed (accuracy, recall, precision, robustness)

    it will use the modules from ./analysis
    prints clearly the result once done

    /!\ (in the future may conduct several at once)

    /!\ don't forget to import those modules in main, like metrics.py

    analysis_params dict is used to know what analysis to conduct
    trigger_params dict is used to get the trigger set from triggerset.py
    data_params dict is used to access the dataset to train the model further and try to remove the WM

    Please refer to nomenclature.md on how to fill out the dictionaries
    '''
    #/!\ add modules in import and in the dict under
    # you can add modules here based on the analysis_params with this dict
    # when implementing multiple analysis steps: loop over them
    # this is done by looping over analysis_params["analysis"]
    # the tuples in the list will be changed to ("res", your_res:str) and you can print your_res at the end
    for module,a_args in analysis_params["analysis"]:
        print(module)
        try: analysis[module]
        except: raise Exception("no module found")
        if module in ["metrics"]:
            analysis[module].metric(model, analysis_params)
        elif module in ["accuracy"] and 'accuracy' not in metrics_value:
            data_params1,use_trigger = a_args
            #print(analysis[module].metric(model,data_params,use_trigger))
            metrics_value[module]=analysis[module].metric(model,data_params,use_trigger)#
        elif module in ["precision","recall"]:
            data_params1,use_trigger = a_args
            #print(analysis[module].metric(model,data_params,use_trigger))
            metrics_value[module]=analysis[module].metric(model,data_params,use_trigger)
        # elif module in ["confusion_matrix"]:
        #     data_params1,use_trigger = a_args
        #     print(analysis[module].metric(model,data_params,use_trigger))
        #     metrics_value[module]=analysis[module].metric(model,data_params,use_trigger)
        # else:
        #     raise NotImplementedError("analysis module behavior not defined in result")
# helper functions

# def get_dataset(data_params:dict) -> tbd:
#     '''
#     this is mainly for testing

#     Calls dataset.py to retrieve dataset (train/test or both ?? TBD)
#     '''
#     #dataset.get_dataset(data_params) #should be enough here
#     raise NotImplementedError()

# def get_model(model_params:dict, trainset:tbd) -> tf.keras.Model:
#     '''
#     this is mainly for testing

#     Calls ./models/classifiers/module.py to
#     retrieve the model
#     '''
#     #the module to use is in models_params
#     raise NotImplementedError()

#to copy for new function
def func(param:type) -> None:
    ''' docstring '''
    raise NotImplementedError()

In [22]:
main(model_params=model_params,
    data_params=data_params,
    analysis_params=analysis_params)
# in metrics : categories if cifar-100 ???
init=False
if init:
    fineT=pd.DataFrame(columns=list(hyperparams.keys())+list(metrics_value.keys()),
                    index=[1],
                    data=np.array(list(hyperparams.values())+list(metrics_value.values())).reshape(1,-1))
    fineT.to_csv('finit.csv')
else:
    fineT=pd.read_csv('finit.csv',index_col=0)
    n=fineT.index[-1] +1
    new_row=pd.DataFrame(columns=list(hyperparams.keys())+list(metrics_value.keys()),
                        index=[n],
                        data=np.array(list(hyperparams.values())+list(metrics_value.values())).reshape(1,-1))
    fineT=pd.concat([fineT,new_row],axis=0)
    # #display(fineT)
    # display(new_row)
    # display(pd.concat([fineT,new_row],axis=0))
    fineT.to_csv('finit.csv')

#print(metrics_value)

print("all done")

Epoch 1/10
1250/1250 [==============================] - 73s 58ms/step - loss: 2.1037 - accuracy: 0.3588
Epoch 2/10
1250/1250 [==============================] - 52s 41ms/step - loss: 1.5458 - accuracy: 0.4422
Epoch 3/10
1250/1250 [==============================] - 43s 34ms/step - loss: 1.4778 - accuracy: 0.4691
Epoch 4/10
1250/1250 [==============================] - 41s 33ms/step - loss: 1.4572 - accuracy: 0.4813
Epoch 5/10
1250/1250 [==============================] - 42s 34ms/step - loss: 1.4150 - accuracy: 0.4975
Epoch 6/10
1250/1250 [==============================] - 42s 33ms/step - loss: 1.3886 - accuracy: 0.5073
Epoch 7/10
1250/1250 [==============================] - 41s 32ms/step - loss: 1.3672 - accuracy: 0.5138
Epoch 8/10
1250/1250 [==============================] - 43s 34ms/step - loss: 1.3423 - accuracy: 0.5267
Epoch 9/10
1250/1250 [==============================] - 52s 41ms/step - loss: 1.3376 - accuracy: 0.5296
Epoch 10/10
1250/1250 [==============================] - 44s 35m

/var/folders/23/vqsnnznx7wv5flx650yb2x9w0000gp/T/ipykernel_1443/1697661206.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  data=np.array(list(hyperparams.values())+list(metrics_value.values())).reshape(1,-1))


In [14]:
print(metrics_value)
display(fineT)
display(new_row)

{'accuracy': 0.92, 'precision': ({'airplane': 0.7367003367003367, 'automobile': 0.691543438077634, 'bird': 0.5936564931178935, 'cat': 0.3946256548926821, 'deer': 0.6368192449047778, 'dog': 0.5909090909090909, 'frog': 0.6807468625650444, 'horse': 0.7187827911857293, 'ship': 0.49328807282826725, 'truck': 0.5591299579601535}, 0.6096201943141609), 'recall': ({'airplane': 0.27119484382746656, 'automobile': 0.743417784401391, 'bird': 0.5007571933366987, 'cat': 0.5723039215686274, 'deer': 0.4755489021956088, 'dog': 0.4308310318855134, 'frog': 0.559496855345912, 'horse': 0.7052767052767053, 'ship': 0.8071194142893209, 'truck': 0.7486539402838962}, 0.581460059241114)}


,train_ratio,val_ration,test_ration,batch_size,nb_epochs,learning_rate,archi,kernel_size,activation,nb_targets,nb_layers,add_pooling,pooling_size,nb_units,optimizer,loss,accuracy,precision,recall
1,0.5,0.3,0.2,32,10,0.003,convo,"(3, 3)",relu,10,2,True,"(2, 2)","[32, 64]",<class 'keras.optimizers.optimizer_v2.adam.Adam'>,sparse_categorical_crossentropy,1.0,"({'airplane': 0.8888888888888888, 'automobile'...","({'airplane': 0.0019831432821021317, 'automobi..."
2,0.5,0.3,0.2,32,10,0.003,dense,"(3, 3)",relu,10,2,True,"(2, 2)","[32, 64]",<class 'keras.optimizers.optimizer_v2.adam.Adam'>,sparse_categorical_crossentropy,0.0,"({'airplane': 0, 'automobile': 0, 'bird': 0, '...","({'airplane': 0.0, 'automobile': 0.0, 'bird': ..."
3,0.5,0.3,0.2,32,10,0.003,convo,"(3, 3)",relu,10,3,True,"(2, 2)","[32, 64, 64]",<class 'keras.optimizers.optimizer_v2.adam.Adam'>,sparse_categorical_crossentropy,0.08,"({'airplane': 0.773444976076555, 'automobile':...","({'airplane': 0.801437778879524, 'automobile':..."
4,0.5,0.3,0.2,32,10,0.003,convo,"(3, 3)",relu,10,3,True,"(2, 2)","[32, 64, 10]",<class 'keras.optimizers.optimizer_v2.adam.Adam'>,sparse_categorical_crossentropy,0.0,"({'airplane': 0.10085, 'automobile': 0, 'bird'...","({'airplane': 1.0, 'automobile': 0.0, 'bird': ..."
5,0.5,0.3,0.2,32,10,0.003,dense,"(3, 3)",relu,10,3,True,"(2, 2)","[32, 64, 10]",<class 'keras.optimizers.optimizer_v2.adam.Adam'>,sparse_categorical_crossentropy,0.0,"({'airplane': 0, 'automobile': 0, 'bird': 0, '...","({'airplane': 0.0, 'automobile': 0.0, 'bird': ..."
6,0.5,0.3,0.2,32,10,0.003,convo,"(3, 3)",relu,10,3,True,"(2, 2)","[32, 32, 32]",<class 'keras.optimizers.optimizer_v2.adam.Adam'>,sparse_categorical_crossentropy,0.08,"({'airplane': 0.7802060568217296, 'automobile'...","({'airplane': 0.6194843827466534, 'automobile'..."
7,0.5,0.3,0.2,32,10,0.003,convo,"(3, 3)",relu,10,3,True,"(2, 2)","[32, 16, 16]",<class 'keras.optimizers.optimizer_v2.adam.Adam'>,sparse_categorical_crossentropy,0.0,"({'airplane': 0, 'automobile': 0.10065, 'bird'...","({'airplane': 0.0, 'automobile': 1.0, 'bird': ..."
8,0.5,0.3,0.2,32,10,0.003,convo,"(3, 3)",relu,10,1,True,"(2, 2)",[64],<class 'keras.optimizers.optimizer_v2.adam.Adam'>,sparse_categorical_crossentropy,0.04,"({'airplane': 0.7819510965465087, 'automobile'...","({'airplane': 0.7689638076351016, 'automobile'..."
9,0.5,0.3,0.2,32,10,0.003,convo,"(3, 3)",relu,10,1,True,"(2, 2)",[96],<class 'keras.optimizers.optimizer_v2.adam.Adam'>,sparse_categorical_crossentropy,0.08,"({'airplane': 0.8773965691220988, 'automobile'...","({'airplane': 0.8621715418939019, 'automobile'..."
10,0.5,0.3,0.2,32,10,0.003,convo,"(3, 3)",relu,10,2,True,"(2, 2)","[32, 32]",<class 'keras.optimizers.optimizer_v2.adam.Adam'>,sparse_categorical_crossentropy,0.12,"({'airplane': 0.7533440342429106, 'automobile'...","({'airplane': 0.6980664352999504, 'automobile'..."


,train_ratio,val_ration,test_ration,batch_size,nb_epochs,learning_rate,archi,kernel_size,activation,nb_targets,nb_layers,add_pooling,pooling_size,nb_units,optimizer,loss,accuracy,precision,recall
18,0.5,0.3,0.2,32,10,0.003,convo,"(3, 3)",relu,10,3,True,"(2, 2)","[32, 64, 128]",<class 'keras.optimizers.optimizer_v2.adam.Adam'>,sparse_categorical_crossentropy,0.92,"({'airplane': 0.7367003367003367, 'automobile'...","({'airplane': 0.27119484382746656, 'automobile..."
